In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import chirp, butter, lfilter
import h5py


fs = 20e6           # Sample rate (20 MSPS, matches PlutoSDR SP1.1)
fc = 2.45e9         # Center frequency (ISM band) [currently not used]
signal_duration = 0.1  # 100 ms signal blocks; consider reducing if memory issues occur
snr_db = 10         # Signal-to-noise ratio (adjust for realism)


def generate_drone_signal(t):
    """Simulate a drone-like modulated signal (e.g., QPSK-like)"""
    # Baseband modulation (simplified)
    symbols = np.random.randint(0, 4, size=len(t))  # 4-QAM symbols
    i = np.cos(2 * np.pi * 1e6 * t) * (symbols % 2)
    q = np.sin(2 * np.pi * 1e6 * t) * ((symbols // 2) % 2)
    signal = i + 1j * q
    
    # Optional: shift to passband using the center frequency
    # signal = signal * np.exp(1j * 2 * np.pi * fc * t)
    
    return signal

def add_noise(signal, snr_db):
    """Add Gaussian noise at specified SNR"""
    signal_power = np.mean(np.abs(signal)**2)
    noise_power = signal_power / (10 ** (snr_db / 10))
    noise = np.sqrt(noise_power) * (np.random.randn(*signal.shape) + 1j*np.random.randn(*signal.shape))/np.sqrt(2)
    return signal + noise

def apply_freq_offset(signal, fs, offset_hz):
    """Simulate frequency offset (e.g., from oscillator drift)"""
    t = np.arange(len(signal)) / fs
    return signal * np.exp(1j * 2 * np.pi * offset_hz * t)


num_samples = 1000  # Number of signal blocks
data = []
labels = []

for _ in range(num_samples):
    t = np.arange(int(signal_duration * fs)) / fs
    
    # Randomly choose class (drone or noise)
    if np.random.rand() < 0.5:  # 50% drone signals
        iq = generate_drone_signal(t)
        iq = apply_freq_offset(iq, fs, offset_hz=np.random.uniform(-1e3, 1e3)) 
        label = 1 
    else:
        iq = np.random.randn(len(t)) + 1j * np.random.randn(len(t))  
        label = 0  # Noise
        
    iq = add_noise(iq, snr_db)
    data.append(iq)
    labels.append(label)


data = np.array(data)
labels = np.array(labels)


with h5py.File('synthetic_drone_data.h5', 'w') as f:
    f.create_dataset('iq_data', data=data)
    f.create_dataset('labels', data=labels)
    f.attrs['fs'] = fs
    f.attrs['description'] = 'Synthetic drone RF signals with noise'

print(f"Saved {num_samples} samples to synthetic_drone_data.h5")


MemoryError: Unable to allocate 29.8 GiB for an array with shape (1000, 2000000) and data type complex128

In [9]:
import os
os.environ["IIO_URI"] = "local:"  # Set a dummy URI

from unittest.mock import MagicMock
import adi

# Override the context manager or the parts that fail
adi.context_manager.context_manager = MagicMock(return_value=None)


TypeError: argument of type 'NoneType' is not iterable